<a href="https://colab.research.google.com/github/96jonesa/CSE-517-Project/blob/main/testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Imports

In [1]:
!pip3 install --quiet "tensorflow-hub>=0.7.0"
!pip3 install --quiet seaborn
!pip3 install --quiet pandas-market-calendars

     |████████████████████████████████| 61kB 7.9MB/s 
     |████████████████████████████████| 112kB 22.0MB/s 


In [2]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from absl import logging
import tensorflow.compat.v1 as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import re
import seaborn as sns
import json
import itertools
import pandas as pd
from tqdm import tqdm_notebook

#Layers

In [3]:
class GRU(nn.Module):
    def __init__(self, input_size, hidden_size, batch_first=False):
        super(GRU, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.batch_first = batch_first

        self.gru = nn.GRU(input_size, hidden_size, batch_first=self.batch_first)

    def forward(self, input):
        output, hn = self.gru(input)
        return output, hn

In [124]:
# attention weights are softmax(u^T tanh(W input + b)) where W is learned parameter matrix, u is a learned parameter vector, and b is a learned offset

class LinearAttention(nn.Module):
    def __init__(self, input_size, intermediate_size, weights_size):
        super(LinearAttention, self).__init__()
        self.input_size = input_size
        self.intermediate_size = intermediate_size
        self.weights_size = weights_size

        self.linear_1 = nn.Linear(self.input_size, self.intermediate_size, bias=True)
        self.linear_2 = nn.Linear(self.intermediate_size, self.weights_size, bias=False)
        self.tanh = nn.Tanh()
        self.softmax = nn.Softmax(dim=2)

    def forward(self, input, mask=None):
        intermediate = self.tanh(self.linear_1(input))
        pre_attention = self.linear_2(intermediate)
        if mask is not None:
            zero_vec = -9e15*torch.ones_like(pre_attention)
            pre_attention = torch.where(mask > 0, pre_attention, zero_vec)
        attention_weights = self.softmax(pre_attention)
        attention_weights = attention_weights.permute(0, 2, 1)
        output_features = torch.bmm(attention_weights, input)

        return output_features

In [5]:
# output is ReLU(left^T W right + b) where W is a learned paramater matrix
# and b is a learned bias

class Blend(nn.Module):
    def __init__(self, left_size, right_size, output_size):
        super(Blend, self).__init__()
        self.left_size = left_size
        self.right_size = right_size
        self.output_size = output_size

        self.bilinear = nn.Bilinear(self.left_size, self.right_size, output_size, bias=True)
        self.relu = nn.ReLU()
    
    def forward(self, left, right):
        output = self.relu(self.bilinear(left, right))

        return output

In [6]:
# https://github.com/Diego999/pyGAT/blob/master/layers.py

class SGAT(nn.Module):
    def __init__(self, input_size, output_size, leakyrelu_slope=0.01):
        super(SGAT, self).__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.leakyrelu_slope = leakyrelu_slope
        
        self.W = nn.Parameter(torch.empty(size=(input_size, output_size)))
        nn.init.xavier_uniform_(self.W.data, gain=1.414)
        self.a = nn.Parameter(torch.empty(size=(2*output_size, 1)))
        nn.init.xavier_uniform_(self.a.data, gain=1.414)
        self.leakyrelu = nn.LeakyReLU(self.leakyrelu_slope)

    def forward(self, h, adj):
        Wh = torch.mm(h, self.W)
        a_input = self._prepare_attentional_mechanism_input(Wh)
        e = self.leakyrelu(torch.matmul(a_input, self.a).squeeze(2))

        zero_vec = -9e15*torch.ones_like(e)
        attention = torch.where(adj > 0, e, zero_vec)
        attention = F.softmax(attention, dim=1)
        h_prime = torch.matmul(attention, Wh)

        return h_prime

    def _prepare_attentional_mechanism_input(self, Wh):
        N = Wh.size()[0] # number of nodes
        
        Wh_repeated_in_chunks = Wh.repeat_interleave(N, dim=0)
        Wh_repeated_alternating = Wh.repeat(N, 1)

        all_combinations_matrix = torch.cat([Wh_repeated_in_chunks, Wh_repeated_alternating], dim=1)

        return all_combinations_matrix.view(N, N, 2 * self.output_size)

In [125]:
class MANSF(nn.Module):
    def __init__(self, T, num_stocks, gru_hidden_size, attn_inter_size, use_embed_size,
                 blend_size, gat_1_inter_size, gat_2_inter_size, leakyrelu_slope, elu_alpha, U):
        super(MANSF, self).__init__()
        self.T = T
        self.num_stocks = num_stocks
        self.gru_hidden_size = gru_hidden_size
        self.attn_inter_size = attn_inter_size
        self.use_embed_size = use_embed_size
        self.blend_size = blend_size
        self.gat_1_inter_size = gat_1_inter_size
        self.gat_2_inter_size = gat_2_inter_size
        self.leakyrelu_slope = leakyrelu_slope
        self.elu_alpha = elu_alpha
        self.U = U

        self.gru_p = GRU(3, gru_hidden_size, batch_first=True)
        self.gru_m = GRU(use_embed_size, gru_hidden_size, batch_first=True)
        self.gru_s = GRU(gru_hidden_size, gru_hidden_size, batch_first=True)
        self.attn_p = LinearAttention(gru_hidden_size, attn_inter_size, 1)
        self.attn_m = LinearAttention(gru_hidden_size, attn_inter_size, 1)
        self.attn_s = LinearAttention(gru_hidden_size, attn_inter_size, 1)
        self.blend = Blend(gru_hidden_size, gru_hidden_size, blend_size)
        self.mgat_1 = nn.ModuleList([SGAT(blend_size, gat_1_inter_size, leakyrelu_slope=leakyrelu_slope) for u in range(U)])
        self.mgat_2 = nn.ModuleList([SGAT(U * gat_1_inter_size, gat_2_inter_size, leakyrelu_slope=leakyrelu_slope) for u in range(U)])
        self.sigmoid = nn.Sigmoid()
        self.elu = nn.ELU(elu_alpha)
        self.final_linear = nn.Linear(U * gat_2_inter_size, 1, bias=True)

    # p is price data tensor of shape (num_stocks, T, 3), for the day under consideration
    # m is smi data list of tensors of shape (num_stocks, K, use_embed_size) of length T,
    #       where K is the number of tweets for the given stock on the day under consideration
    # neighorhoods is a list of adjacency lists, where each stock is indexed with the same
    #       indices they have in p and m
    def forward(self, p, m, m_mask, neighborhoods):
        ## price encoding
        h_p, _ = self.gru_p(p)
        q = self.attn_p(h_p)

        ## smi encoding (day level)
        r = torch.zeros(self.num_stocks, 0, self.gru_hidden_size)
        r = r.to(device)
        for t in range(self.T):
            h_m, _ = self.gru_m(m[t])
            r_t = self.attn_m(h_m, m_mask[t])
            r = torch.cat((r, r_t), 1)

        ## smi encoding (aggregate)
        h_s, _ = self.gru_s(r)
        c = self.attn_s(h_s)

        ## blending
        x = self.blend(q, c)

        ## reshaping (eliminating superfluous dimension)
        x = x.view(x.shape[0], x.shape[2])

        ## first gat layer
        #  first head
        sgat = self.mgat_1[0]
        z = sgat(x, neighborhoods)
        z = self.elu(z)

        #  remaining heads
        for u in range(1, self.U):
            sgat = self.mgat_1[u]
            z_u = sgat(x, neighborhoods)
            z_u = self.elu(z_u)
            
            z = torch.cat((z, z_u), 1)
        
        ## second gat layer
        #  first head
        sgat = self.mgat_2[0]
        new_z = sgat(z, neighborhoods)
        new_z = self.sigmoid(new_z)

        #  remaining heads
        for u in range(1, self.U):
            sgat = self.mgat_2[u]
            new_z_u = sgat(z, neighborhoods)
            new_z_u = self.sigmoid(new_z_u)
            
            new_z = torch.cat((new_z, new_z_u), 1)
        
        ## final layer
        y = self.sigmoid(self.final_linear(new_z))

        ## return result
        return y

#Data Processing

In [8]:
!wget https://github.com/yumoxu/stocknet-dataset/archive/master.zip

--2021-03-11 08:50:05--  https://github.com/yumoxu/stocknet-dataset/archive/master.zip
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/yumoxu/stocknet-dataset/zip/master [following]
--2021-03-11 08:50:05--  https://codeload.github.com/yumoxu/stocknet-dataset/zip/master
Resolving codeload.github.com (codeload.github.com)... 140.82.114.9
Connecting to codeload.github.com (codeload.github.com)|140.82.114.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘master.zip’

master.zip              [    <=>             ]  93.77M  9.36MB/s    in 13s     

2021-03-11 08:50:18 (7.32 MB/s) - ‘master.zip’ saved [98325732]



In [9]:
!unzip master.zip

Streaming output truncated to the last 5000 lines.
  inflating: stocknet-dataset-master/tweet/raw/T/2014-03-01  
  inflating: stocknet-dataset-master/tweet/raw/T/2014-03-02  
  inflating: stocknet-dataset-master/tweet/raw/T/2014-03-03  
  inflating: stocknet-dataset-master/tweet/raw/T/2014-03-04  
  inflating: stocknet-dataset-master/tweet/raw/T/2014-03-05  
  inflating: stocknet-dataset-master/tweet/raw/T/2014-03-06  
  inflating: stocknet-dataset-master/tweet/raw/T/2014-03-07  
  inflating: stocknet-dataset-master/tweet/raw/T/2014-03-08  
  inflating: stocknet-dataset-master/tweet/raw/T/2014-03-09  
  inflating: stocknet-dataset-master/tweet/raw/T/2014-03-10  
  inflating: stocknet-dataset-master/tweet/raw/T/2014-03-11  
  inflating: stocknet-dataset-master/tweet/raw/T/2014-03-12  
  inflating: stocknet-dataset-master/tweet/raw/T/2014-03-13  
  inflating: stocknet-dataset-master/tweet/raw/T/2014-03-14  
  inflating: stocknet-dataset-master/tweet/raw/T/2014-03-15  
  inflating: stockn

In [10]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/2" #@param ["https://tfhub.dev/google/universal-sentence-encoder/2", "https://tfhub.dev/google/universal-sentence-encoder-large/3"]

In [11]:
tf.disable_v2_behavior()
tf.compat.v1.disable_eager_execution()

Instructions for updating:
non-resource variables are not supported in the long term


In [12]:
stocknet_dataset_filepath = './stocknet-dataset-master'
start_date = '2014-01-01'
end_date = '2016-01-01'

In [13]:
preprocessed_prices_filepath = stocknet_dataset_filepath + '/price/preprocessed'
preprocessed_tweets_filepath = stocknet_dataset_filepath + '/tweet/preprocessed'

company_to_price_df = {}
company_to_tweets = {}

for filename in os.listdir(preprocessed_prices_filepath):
    with open(preprocessed_prices_filepath + '/' + filename) as file:
        company_name = filename.split('.')[0]
        
        # Not enough data for GMRE
        if company_name == 'GMRE':
            continue
        df = pd.read_csv(file, sep='\t')
        df.columns = ['date', 'open', 'high', 'low', 'close', 'adjust_close', 'volume']
        company_to_price_df[company_name] = df.dropna()

for filename in tqdm_notebook(os.listdir(preprocessed_tweets_filepath)):
    company_name = filename.split('.')[0]
    dates_to_tweets = {}
    for tweet_filename in os.listdir(preprocessed_tweets_filepath + '/' + filename):
        with open(preprocessed_tweets_filepath + '/' + filename + '/' + tweet_filename) as file:
            list_of_tweets = []
            for line in file:
                tweet_json = json.loads(line)
                list_of_tweets.append(tweet_json)
            dates_to_tweets[tweet_filename] = list_of_tweets
    company_to_tweets[company_name] = dates_to_tweets

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


In [14]:
# Reduce logging output.
logging.set_verbosity(logging.ERROR)
tf.get_logger().setLevel(logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

# Import the Universal Sentence Encoder's TF Hub module
def embed_useT(module):
    with tf.Graph().as_default():
        sentences = tf.placeholder(tf.string)
        embed = hub.Module(module)
        embeddings = embed(sentences)
        session = tf.train.MonitoredSession()
    return lambda x: session.run(embeddings, {sentences: x})
embed_fn = embed_useT(module_url)

In [15]:
# Generate embeddings
for company in tqdm_notebook(company_to_tweets.keys()):
  for date in company_to_tweets[company].keys():
    messages = []
    for j in range(len(company_to_tweets[company][date])):
      messages.append(' '.join(company_to_tweets[company][date][j]['text']))
    message_embeddings = embed_fn(messages)
    for k in range(len(company_to_tweets[company][date])):
      company_to_tweets[company][date][k]['embedding'] = list(message_embeddings[k])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [16]:
# Create date mapping
date_universe = set()
for company in company_to_price_df.keys():
    date_universe = date_universe.union(set(company_to_price_df[company].date))
for company in company_to_tweets.keys():
    date_universe = date_universe.union(set(company_to_tweets[company].keys()))
date_universe = sorted(list(date_universe))
index_to_date = {i-5:d for i,d in enumerate(date_universe)}
date_to_index = {d:i-5 for i,d in enumerate(date_universe)}

In [17]:
# Calculate dimensions for tensor
n_stocks = len(company_to_tweets.keys())
n_days = len(date_universe)
max_tweets = 0
for c,d in itertools.product(company_to_tweets.keys(), date_universe):
    if d in company_to_tweets[c]:
        max_tweets = max(max_tweets, len(company_to_tweets[c][d]))
# Create index mapping for stocks alphabetically
company_to_index = {c:i for i,c in enumerate(sorted(list(company_to_tweets.keys())))}
# print dimensions
print(n_stocks)
print(n_days)
print(max_tweets)

87
1473
555


In [332]:
n_days = 8

In [333]:
# Construct tensors
price_tensor = np.zeros((n_stocks, n_days - 5, 6, 3))
smi_tensor = np.zeros((n_stocks, n_days - 5, 6, max_tweets, 512))

In [334]:
# price tensor
for company in tqdm_notebook(company_to_price_df.keys()):
    dates = sorted(list(company_to_price_df[company].date))
    lags = []
    for date in dates[:5]:
        entry = []
        row = company_to_price_df[company].loc[company_to_price_df[company]['date'] == date]
        entry.append(row['high'].values[0])
        entry.append(row['low'].values[0])
        entry.append(row['adjust_close'].values[0])
        lags.append(entry)
    date_index = 0
    for date in dates[5:n_days]:
        entry = []
        row = company_to_price_df[company].loc[company_to_price_df[company]['date'] == date]
        entry.append(row['high'].values[0])
        entry.append(row['low'].values[0])
        entry.append(row['adjust_close'].values[0])
        lags.append(entry)
        company_index = company_to_index[company]
        for i,entry in enumerate(lags):
            for j,price in enumerate(entry):
                #stocks, day, lags, hi/lo/close
                price_tensor[company_index, date_index, i, j] = price
        lags.pop(0)
        date_index += 1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [335]:
# SMI tensor
for company in tqdm_notebook(company_to_tweets.keys()):
    dates = sorted(list(company_to_tweets[company].keys()))
    lags = []
    for date in dates[:5]:
        n_tweets = len(company_to_tweets[company][date])
        lags.append([company_to_tweets[company][date][k]['embedding'] for k in range(n_tweets)])
    date_index = 0
    for date in dates[5:n_days]:
        n_tweets = len(company_to_tweets[company][date])
        lags.append([company_to_tweets[company][date][k]['embedding'] for k in range(n_tweets)])
        company_index = company_to_index[company]
        for i,messages in enumerate(lags):
            for j,embedding in enumerate(messages):
                #stocks, day, lags, tweet, embedding
                smi_tensor[company_index, date_index, i, j, :] = embedding[:]
        lags.pop(0)
        date_index += 1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


#Separator

In [336]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [337]:
price_tensor = torch.from_numpy(price_tensor)
smi_tensor = torch.from_numpy(smi_tensor)

price_tensor = price_tensor.type(torch.FloatTensor)
smi_tensor = smi_tensor.type(torch.FloatTensor)

price_tensor = price_tensor.to(device)
smi_tensor = smi_tensor.to(device)

In [338]:
price_tensor.shape

torch.Size([87, 3, 6, 3])

In [339]:
price_tensor = price_tensor.permute(1, 0, 2, 3)

In [340]:
p = price_tensor[0]

In [341]:
p_ = price_tensor[1]

In [342]:
p__ = price_tensor[2]

In [343]:
p.shape

torch.Size([87, 6, 3])

In [344]:
smi_tensor.shape

torch.Size([87, 3, 6, 555, 512])

In [345]:
smi_tensor = smi_tensor.permute(1, 2, 0, 3, 4)

In [346]:
m = []
for t in range(6):
    m.append(smi_tensor[0][t])

In [347]:
m_ = []
for t in range(6):
    m_.append(smi_tensor[1][t])

In [348]:
m__ = []
for t in range(6):
    m__.append(smi_tensor[2][t])

In [349]:
neighborhoods = torch.eye(n_stocks, n_stocks)
neighborhoods = neighborhoods.to(device)

In [350]:
m_K = torch.ones(6, n_stocks)

In [351]:
m_mask = torch.zeros(6, n_stocks, max_tweets, 1)
for t in range(6):
    for i in range(n_stocks):
        for k in range(max_tweets):
            if k <= m_K[t][i]:
                m_mask[t][i][k][0] = 1

m_mask = m_mask.to(device)

#Training

In [352]:
mansf = MANSF(T=6,
              num_stocks=n_stocks,
              gru_hidden_size=64,
              attn_inter_size=32,
              use_embed_size=512,
              blend_size=32,
              gat_1_inter_size=32,
              gat_2_inter_size=32,
              leakyrelu_slope=0.01,
              elu_alpha=1.0,
              U=8)

In [353]:
mansf = mansf.to(device)

In [354]:
y = mansf(p, m, m_mask, neighborhoods)

In [355]:
print(y)  # initial predictions without training

tensor([[0.4140],
        [0.3730],
        [0.3982],
        [0.3838],
        [0.4217],
        [0.4142],
        [0.3740],
        [0.4041],
        [0.3881],
        [0.4065],
        [0.3421],
        [0.3537],
        [0.3509],
        [0.3471],
        [0.3163],
        [0.3723],
        [0.4058],
        [0.4006],
        [0.3476],
        [0.4227],
        [0.3764],
        [0.4144],
        [0.4068],
        [0.3338],
        [0.3943],
        [0.3603],
        [0.3865],
        [0.4175],
        [0.3994],
        [0.3820],
        [0.3860],
        [0.4069],
        [0.3983],
        [0.3830],
        [0.3660],
        [0.4075],
        [0.3887],
        [0.4319],
        [0.3436],
        [0.3348],
        [0.3994],
        [0.3913],
        [0.3793],
        [0.3918],
        [0.3807],
        [0.4099],
        [0.3801],
        [0.4051],
        [0.4125],
        [0.3809],
        [0.3846],
        [0.3969],
        [0.3956],
        [0.3731],
        [0.3844],
        [0

In [356]:
Y = torch.zeros(n_stocks, 1)

for i in range(n_stocks):
    if p_[i][5][2] > 0:
        Y[i][0] = 1.0

In [357]:
Y = Y.to(device)

In [358]:
optimizer = optim.Adam(mansf.parameters())
loss_fn = nn.BCELoss()

In [359]:
for epoch in range(100):
    mansf.train()
    y = mansf(p, m, m_mask, neighborhoods)
    loss = loss_fn(y.view(-1), Y.view(-1))
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    print(epoch, loss.item())

0 0.9257104992866516
1 0.5476553440093994
2 0.4284573197364807
3 0.34990090131759644
4 0.30110159516334534
5 0.273024320602417
6 0.2547696530818939
7 0.24097304046154022
8 0.2312682718038559
9 0.22564013302326202
10 0.22216984629631042
11 0.22037260234355927
12 0.22094713151454926
13 0.2212858945131302
14 0.22143423557281494
15 0.22166071832180023
16 0.22169633209705353
17 0.22145181894302368
18 0.22101803123950958
19 0.22101671993732452
20 0.2213541716337204
21 0.2215534895658493
22 0.22151346504688263
23 0.22121937572956085
24 0.22079546749591827
25 0.22052764892578125
26 0.22047647833824158
27 0.22030314803123474
28 0.21999455988407135
29 0.21968139708042145
30 0.21940353512763977
31 0.21902251243591309
32 0.21851934492588043
33 0.21807861328125
34 0.2178124636411667
35 0.21751530468463898
36 0.217162624001503
37 0.21668243408203125
38 0.21629004180431366
39 0.2159782499074936
40 0.2156507521867752
41 0.21528993546962738
42 0.21497642993927002
43 0.21474285423755646
44 0.21435537934

In [360]:
ycatY = torch.cat((y, Y), dim=1)

In [361]:
print(ycatY)

tensor([[0.9596, 1.0000],
        [0.9505, 1.0000],
        [0.9588, 1.0000],
        [0.9569, 1.0000],
        [0.8856, 0.0000],
        [0.9586, 1.0000],
        [0.9801, 1.0000],
        [0.9576, 1.0000],
        [0.9662, 1.0000],
        [0.9466, 1.0000],
        [0.9730, 1.0000],
        [0.9075, 0.0000],
        [0.9728, 1.0000],
        [0.9206, 1.0000],
        [0.9801, 1.0000],
        [0.9581, 1.0000],
        [0.9594, 1.0000],
        [0.9587, 1.0000],
        [0.9745, 1.0000],
        [0.9074, 1.0000],
        [0.9562, 1.0000],
        [0.8849, 1.0000],
        [0.9547, 1.0000],
        [0.9126, 1.0000],
        [0.9573, 1.0000],
        [0.9637, 1.0000],
        [0.9570, 1.0000],
        [0.9155, 1.0000],
        [0.9516, 1.0000],
        [0.9579, 1.0000],
        [0.9584, 1.0000],
        [0.8812, 0.0000],
        [0.9554, 1.0000],
        [0.9538, 1.0000],
        [0.9810, 1.0000],
        [0.9571, 1.0000],
        [0.9537, 1.0000],
        [0.9328, 1.0000],
        [0.9

In [362]:
accuracy = torch.sum((y > 0.5) == Y) / len(y)

In [363]:
print(accuracy)

tensor(0.9425, device='cuda:0')


In [364]:
y_ = mansf(p_, m_, m_mask, neighborhoods)

In [365]:
Y_ = torch.zeros(n_stocks, 1)

for i in range(n_stocks):
    if p__[i][5][2] > 0:
        Y_[i][0] = 1.0

In [366]:
Y_ = Y_.to(device)

In [367]:
ycatY_ = torch.cat((y_, Y_), dim=1)

In [368]:
print(ycatY_)

tensor([[0.9554, 1.0000],
        [0.9454, 1.0000],
        [0.9587, 1.0000],
        [0.9553, 0.0000],
        [0.8839, 0.0000],
        [0.9590, 0.0000],
        [0.9726, 1.0000],
        [0.9589, 0.0000],
        [0.9601, 0.0000],
        [0.9475, 1.0000],
        [0.9744, 1.0000],
        [0.9129, 0.0000],
        [0.9744, 1.0000],
        [0.9107, 1.0000],
        [0.9766, 1.0000],
        [0.9587, 1.0000],
        [0.9608, 1.0000],
        [0.9686, 1.0000],
        [0.9743, 1.0000],
        [0.9045, 1.0000],
        [0.9542, 1.0000],
        [0.8733, 1.0000],
        [0.9534, 1.0000],
        [0.9116, 1.0000],
        [0.9573, 1.0000],
        [0.9727, 1.0000],
        [0.9560, 0.0000],
        [0.9121, 0.0000],
        [0.9489, 0.0000],
        [0.9575, 0.0000],
        [0.9582, 1.0000],
        [0.8821, 1.0000],
        [0.9558, 1.0000],
        [0.9537, 1.0000],
        [0.9791, 1.0000],
        [0.9536, 1.0000],
        [0.9694, 0.0000],
        [0.9252, 0.0000],
        [0.9

In [369]:
accuracy_ = torch.sum((y_ > 0.5) == Y_) / len(y_)

In [370]:
print(accuracy_)

tensor(0.6092, device='cuda:0')
